<a href="https://colab.research.google.com/github/yonikremer/final_project/blob/master/profiling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook should be run in colab

In [3]:
import os, shutil, sys

In [8]:
if os.path.exists("/content/final_project") and os.path.isdir("/content/final_project"):
    shutil.rmtree("/content/final_project")
os.system("git clone https://github.com/yonikremer/final_project.git");

if not os.path.exists("/content/final_project/evaluation/comet_ml_api_key.txt"):
    with open("/content/final_project/evaluation/comet_ml_api_key.txt", 'w') as f:
        f.write("yNLPmFeYvfF9HArNCZ2xm8RHn")

sys.path.append('/content/final_project');

In [4]:
sys.path.append('/content/final_project');

In [2]:
%%capture
%pip install -q -r /content/final_project/evaluation/evaluation_requirements.txt
%pip install -q -r /content/final_project/project_requirements.txt

In [7]:
from final_project.sampling_generator import SamplingGenerator
g = SamplingGenerator(
        model_name="facebook/opt-125m",
        group_size=8,
        temp=1.0,
        top_k=None,
        top_p=1.0,
        end_of_sentence_stop=True,
        answer_length_multiplier=4,
    )
example_prompt = "What do you do when your firmly held beliefs turn out not to be true? When Casey Gerald's religion failed him, he searched for something new to believe in -- in business, in government, in philanthropy -- but found only false saviors. In this moving talk, Gerald urges us all to question our beliefs and embrace uncertainty."
%prun g(example_prompt)